# *RAG Implementation Fake News Detection:*


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.multiprocessing as mp # Import torch multiprocessing
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sentence_transformers import SentenceTransformer
import faiss # For efficient similarity search
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm.auto import tqdm
import os
import time
import gc # Garbage collector
import traceback # For detailed error printing

# --- Set Environment Variable to Suppress Tokenizer Parallelism Warning ---
# This should be done early, before tokenizers are potentially used by multiprocessing workers
os.environ["TOKENIZERS_PARALLELISM"] = "false"


# --- 1. Configuration ---
class Config:
    """Configuration class for hyperparameters and settings."""
    # File paths (Update these paths if your files are located elsewhere)
    train_file = '../constraint_train.csv'
    val_file = '../constraint_val.csv'
    test_file = '../english_test_with_labels.csv'

    # Model names
    retriever_model_name = 'all-MiniLM-L6-v2' # Efficient sentence transformer for retrieval
    classifier_model_name = 'bert-base-uncased' # Base model for classification ('roberta-base', etc.)

    # RAG parameters
    num_retrieved_docs = 3 # Number of relevant documents to retrieve per input

    # Training parameters
    max_seq_length = 512 # Max token length for combined input (query + retrieved docs)
    batch_size = 8       # Adjust based on GPU memory
    epochs = 3           # Number of training epochs
    learning_rate = 2e-5 # Learning rate for the optimizer
    warmup_steps = 100   # Number of warmup steps for the scheduler
    gradient_accumulation_steps = 1 # Increase if batch size needs to be effectively larger due to memory limits

    # Hardware and reproducibility
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    seed = 42

    # Output directory for saving the best model
    output_dir = "./rag_classifier_model"

    # --- DEBUGGING FLAG ---
    # Set to True to force num_workers=0 and get better error tracebacks
    # Set back to False for normal parallel data loading
    DEBUG_DATALOADER = True # <-- SET THIS TO True FOR DEBUGGING WORKER ERRORS

# --- 2. Data Loading Function ---
def load_data(filepath):
    """
    Loads data from an Excel file, performs basic cleaning, and maps labels.

    Args:
        filepath (str): Path to the Excel file.

    Returns:
        pandas.DataFrame: Loaded and preprocessed data, or None if loading fails.
    """
    if not os.path.exists(filepath):
        print(f"Error: Data file not found: {filepath}")
        return None
    try:
        df = pd.read_excel(filepath)

        # Verify essential columns exist
        if 'tweet' not in df.columns or 'label' not in df.columns:
             print(f"Error: Excel file {filepath} must contain 'tweet' and 'label' columns.")
             return None

        # Drop rows where 'tweet' is NaN and ensure 'tweet' is string type
        df = df.dropna(subset=['tweet'])
        df['tweet'] = df['tweet'].astype(str)

        # Map labels to integers ('real': 1, 'fake': 0)
        df['label'] = df['label'].map({'real': 1, 'fake': 0})

        # Drop rows where label mapping failed (i.e., label was not 'real' or 'fake')
        df = df.dropna(subset=['label'])
        df['label'] = df['label'].astype(int)

        if df.empty:
            print(f"Warning: No valid data loaded from {filepath} after cleaning.")
            return None

        print(f"Loaded {len(df)} samples from {filepath}")
        return df
    except Exception as e:
        print(f"Error loading {filepath}: {e}")
        traceback.print_exc() # Print detailed traceback
        return None

# --- 3. Custom PyTorch Dataset for RAG ---
class RagNewsDataset(Dataset):
    """
    PyTorch Dataset that retrieves relevant documents using a retriever function
    and combines them with the original text for input to the classifier.
    Includes enhanced error handling in __getitem__.
    """
    def __init__(self, texts, labels, tokenizer, max_len, retriever_func, num_retrieved):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.retriever_func = retriever_func
        self.num_retrieved = num_retrieved

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        """
        Retrieves an item, finds context, tokenizes, and returns dict.
        Includes specific try-except blocks for debugging worker errors.
        """
        try:
            original_text = str(self.texts[idx])
            label = int(self.labels[idx])
        except Exception as e:
            print(f"DataLoader Worker Error: Failed to get text/label at index {idx}. Error: {e}")
            # Option 1: Return None or raise an error to stop (if num_workers=0)
            # Option 2: Return a dummy item (might hide errors but allow continuation)
            # For debugging, raising is better if num_workers=0
            if mp.current_process().daemon: # Check if in a worker process
                 print(f"Problematic text (first 100 chars): {str(self.texts[idx])[:100] if idx < len(self.texts) else 'Index out of bounds'}")
                 # Workers should ideally not raise exceptions that crash them,
                 # but returning None might cause issues downstream in collate_fn.
                 # It's often better to fix the root cause found when num_workers=0.
                 # For now, let it potentially crash the worker if data access fails.
                 pass # Or implement dummy return if needed, but not ideal for finding errors
            raise e # Re-raise if in main process (num_workers=0) or let worker crash

        # --- Retrieval Step ---
        retrieved_docs = []
        try:
            retrieved_docs = self.retriever_func(original_text, k=self.num_retrieved)
        except Exception as e:
            print(f"DataLoader Worker Error: Failed during RETRIEVAL for index {idx}. Error: {e}")
            # Print part of the text that caused the retrieval error
            print(f"  Query text (first 100 chars): {original_text[:100]}")
            # Fallback to empty context, but the error is logged
            retrieved_docs = []
            # If debugging with num_workers=0, you might want to raise e here too

        # --- Tokenization Step ---
        try:
            context = " ".join(retrieved_docs)
            combined_text = f"{original_text} [SEP] {context}"

            encoding = self.tokenizer.encode_plus(
                combined_text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt',
            )
        except Exception as e:
            print(f"DataLoader Worker Error: Failed during TOKENIZATION for index {idx}. Error: {e}")
            # Print part of the text that caused the tokenization error
            print(f"  Combined text (first 100 chars): {combined_text[:100]}")
            # If tokenization fails, we cannot return valid tensors.
            # Raising the error is best when num_workers=0.
            # If in a worker, crashing might be unavoidable if we can't produce valid output.
            if not mp.current_process().daemon:
                 raise e # Re-raise if in main process
            else:
                 # What to do in a worker? Returning None often breaks the collate_fn.
                 # Crashing the worker might be the only outcome, leading back to the original error.
                 # This highlights why num_workers=0 is key for debugging this stage.
                 print("FATAL: Cannot proceed with tokenization error in worker.")
                 # To prevent infinite loops, maybe return a dummy tensor of the correct shape?
                 # This is complex and error-prone. Best to fix the root cause.
                 # For now, let it potentially crash.
                 pass


        # --- Return Result ---
        try:
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)
            }
        except Exception as e:
             print(f"DataLoader Worker Error: Failed during final DICT CREATION for index {idx}. Error: {e}")
             # This should be rare if encoding succeeded.
             if not mp.current_process().daemon:
                  raise e
             else:
                  # Let worker crash if it can't form the final dict
                  pass


# --- 4. Evaluation Function ---
# (Keep evaluate_model function as is)
def evaluate_model(model, dataloader, device, loss_fn):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            total_loss += loss.item()
            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    avg_loss = total_loss / len(dataloader) if len(dataloader) > 0 else 0
    accuracy = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary', zero_division=0)
    print(f"\nEvaluation Results:")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall: {recall:.4f}")
    print(f"  F1-Score: {f1:.4f}")
    return avg_loss, accuracy, precision, recall, f1


# --- Main Execution Block ---
if __name__ == "__main__":

    # --- Multiprocessing Setup ---
    # (Keep multiprocessing setup as is)
    try:
        current_start_method = mp.get_start_method(allow_none=True)
        if current_start_method != 'spawn':
             mp.set_start_method('spawn', force=True)
             print(f"Multiprocessing start method set to 'spawn' (was {current_start_method}).")
        else:
            print("Multiprocessing start method already set to 'spawn'.")
    except RuntimeError as e:
        print(f"Note: Could not set multiprocessing start method to 'spawn': {e}")
    except AttributeError:
        print("Note: Unable to check/set multiprocessing start method (possibly older Python/torch version).")


    # --- Seed and Device Setup ---
    # (Keep seed and device setup as is)
    np.random.seed(Config.seed)
    torch.manual_seed(Config.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(Config.seed)
        print(f"CUDA available. Using device: {Config.device}")
    else:
        print(f"CUDA not available. Using device: {Config.device}")

    # --- Retriever Setup ---
    # (Keep retriever setup as is)
    print("\n--- Setting up Retriever ---")
    train_df = None; train_texts = []; retriever_model = None; faiss_index = None
    try:
        train_df = load_data(Config.train_file)
        if train_df is None: raise ValueError("Failed to load training data for retriever.")
        train_texts = train_df['tweet'].tolist()
        print(f"Loading retriever model: {Config.retriever_model_name}...")
        retriever_model = SentenceTransformer(Config.retriever_model_name, device=Config.device)
        print("Retriever model loaded.")
        print(f"Embedding {len(train_texts)} training documents...")
        batch_size_embed = 64
        train_embeddings = retriever_model.encode(train_texts, batch_size=batch_size_embed, convert_to_tensor=True, show_progress_bar=True, device=Config.device)
        print(f"Embeddings shape: {train_embeddings.shape}")
        train_embeddings_cpu = train_embeddings.cpu().numpy().astype(np.float32)
        del train_embeddings; gc.collect()
        if Config.device == torch.device("cuda"): torch.cuda.empty_cache()
        embedding_dim = train_embeddings_cpu.shape[1]
        print(f"Building FAISS index (IndexFlatL2) with dimension {embedding_dim}...")
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        faiss_index.add(train_embeddings_cpu)
        print(f"FAISS index built with {faiss_index.ntotal} vectors.")
    except Exception as e:
        print(f"FATAL: Error during retriever setup: {e}"); traceback.print_exc(); exit(1)

    # --- Retrieval Function Definition ---
    # (Keep retrieve_documents function as is)
    def retrieve_documents(query_text, k=Config.num_retrieved_docs):
        if retriever_model is None or faiss_index is None: return []
        try:
            query_embedding = retriever_model.encode([query_text], convert_to_tensor=True, device=Config.device)
            query_embedding_np = query_embedding.cpu().numpy().astype(np.float32)
            distances, indices = faiss_index.search(query_embedding_np, k)
            retrieved_texts = []
            for i in indices[0]:
                if 0 <= i < len(train_texts): retrieved_texts.append(train_texts[i])
                else: print(f"Warning: Retrieved invalid index {i} during search.")
            return retrieved_texts
        except Exception as e:
             print(f"ERROR in retrieve_documents for query '{query_text[:50]}...': {e}")
             return []


    # --- Load Data & Create DataLoaders ---
    print("\n--- Loading Data and Creating DataLoaders ---")
    val_df, test_df = None, None
    train_dataset, val_dataset, test_dataset = None, None, None
    train_dataloader, val_dataloader, test_dataloader = None, None, None
    classifier_tokenizer = None

    try:
        # Load validation and test data
        val_df = load_data(Config.val_file)
        test_df = load_data(Config.test_file)
        if val_df is None or test_df is None: raise ValueError("Failed to load validation or test data.")

        # Initialize tokenizer
        print(f"Loading classifier tokenizer: {Config.classifier_model_name}...")
        classifier_tokenizer = AutoTokenizer.from_pretrained(Config.classifier_model_name)
        print("Classifier tokenizer loaded.")

        # Create Datasets
        print("Creating datasets...")
        train_dataset = RagNewsDataset(texts=train_df['tweet'].tolist(), labels=train_df['label'].tolist(), tokenizer=classifier_tokenizer, max_len=Config.max_seq_length, retriever_func=retrieve_documents, num_retrieved=Config.num_retrieved_docs)
        val_dataset = RagNewsDataset(texts=val_df['tweet'].tolist(), labels=val_df['label'].tolist(), tokenizer=classifier_tokenizer, max_len=Config.max_seq_length, retriever_func=retrieve_documents, num_retrieved=Config.num_retrieved_docs)
        test_dataset = RagNewsDataset(texts=test_df['tweet'].tolist(), labels=test_df['label'].tolist(), tokenizer=classifier_tokenizer, max_len=Config.max_seq_length, retriever_func=retrieve_documents, num_retrieved=Config.num_retrieved_docs)
        print("Datasets created.")

        # --- Determine number of workers (DEBUGGING CHANGE) ---
        if Config.DEBUG_DATALOADER:
             num_workers = 0 # Force 0 workers for debugging
             print("DEBUG MODE: Setting num_workers = 0 to get detailed error tracebacks.")
        else:
             # Original logic for setting num_workers
             num_workers = 0
             if Config.device == torch.device("cuda"):
                  if mp.get_start_method(allow_none=True) == 'spawn':
                       num_workers = 2 # Or your desired number
                  else:
                       print("Warning: CUDA available but start method is not 'spawn'. Using 0 dataloader workers.")
             print(f"Using {num_workers} dataloader workers.")
        # --- End Debugging Change ---

        pin_memory = (num_workers > 0 and Config.device == torch.device("cuda"))
        print(f"pin_memory set to: {pin_memory}")

        # Create DataLoaders with the determined num_workers
        train_dataloader = DataLoader(train_dataset, batch_size=Config.batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
        val_dataloader = DataLoader(val_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
        test_dataloader = DataLoader(test_dataset, batch_size=Config.batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)
        print("DataLoaders created.")

        if len(train_dataloader) == 0 or len(val_dataloader) == 0 or len(test_dataloader) == 0:
             print("Warning: One or more DataLoaders are empty.")
        else:
            print(f"  Train batches: {len(train_dataloader)}")
            print(f"  Validation batches: {len(val_dataloader)}")
            print(f"  Test batches: {len(test_dataloader)}")

    except Exception as e:
        print(f"FATAL: Error during DataLoader creation: {e}")
        traceback.print_exc()
        exit(1)

    # --- Model Definition ---
    # (Keep model definition as is)
    print("\n--- Initializing Classifier Model ---")
    classifier_model = None
    try:
        print(f"Loading classifier model: {Config.classifier_model_name}...")
        classifier_model = AutoModelForSequenceClassification.from_pretrained(Config.classifier_model_name, num_labels=2)
        classifier_model.to(Config.device)
        print("Classifier model loaded and moved to device.")
    except Exception as e:
        print(f"FATAL: Failed to initialize classifier model: {e}"); traceback.print_exc(); exit(1)


    # --- Training Setup ---
    # (Keep training setup as is)
    print("\n--- Setting up Training Components ---")
    optimizer = None; scheduler = None; loss_fn = None
    if len(train_dataloader) > 0:
        try:
            optimizer = AdamW(classifier_model.parameters(), lr=Config.learning_rate)
            total_steps = len(train_dataloader) * Config.epochs // Config.gradient_accumulation_steps
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=Config.warmup_steps, num_training_steps=total_steps)
            loss_fn = nn.CrossEntropyLoss()
            print("Optimizer, Scheduler, and Loss Function initialized.")
            print(f"Total training steps (considering grad accum): {total_steps}")
        except Exception as e:
            print(f"FATAL: Error during training setup: {e}"); traceback.print_exc(); exit(1)
    else:
        print("FATAL: Training dataloader is empty."); exit(1)


    # --- Training Loop ---
    # (Keep training loop as is)
    print("\n--- Starting Training ---")
    best_val_f1 = -1.0; global_step = 0
    for epoch in range(Config.epochs):
        print(f"\n===== Epoch {epoch + 1}/{Config.epochs} =====")
        start_time = time.time(); classifier_model.train(); total_train_loss = 0
        optimizer.zero_grad()
        progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1} Training", leave=False)
        for step, batch in enumerate(progress_bar):
            try:
                input_ids = batch['input_ids'].to(Config.device)
                attention_mask = batch['attention_mask'].to(Config.device)
                labels = batch['labels'].to(Config.device)
                outputs = classifier_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                if Config.gradient_accumulation_steps > 1: loss = loss / Config.gradient_accumulation_steps
                loss.backward()
                total_train_loss += loss.item() * Config.gradient_accumulation_steps
                if (step + 1) % Config.gradient_accumulation_steps == 0 or (step + 1) == len(train_dataloader):
                    torch.nn.utils.clip_grad_norm_(classifier_model.parameters(), 1.0)
                    optimizer.step(); scheduler.step(); optimizer.zero_grad(); global_step += 1
                    current_loss_scaled = loss.item() * Config.gradient_accumulation_steps
                    progress_bar.set_postfix({'loss': f"{current_loss_scaled:.4f}"})
            except Exception as e:
                print(f"\nError during training step {step} in epoch {epoch + 1}: {e}"); traceback.print_exc()
                print("Attempting to continue training..."); optimizer.zero_grad(); continue
        progress_bar.close()
        epoch_time = time.time() - start_time
        avg_train_loss = total_train_loss / global_step if global_step > 0 else 0
        print(f"\nEpoch {epoch + 1} completed in {epoch_time:.2f} seconds.")
        print(f"  Average Training Loss: {avg_train_loss:.4f}")
        if len(val_dataloader) > 0:
            print("\n--- Evaluating on Validation Set ---")
            val_loss, val_acc, val_prec, val_rec, val_f1 = evaluate_model(classifier_model, val_dataloader, Config.device, loss_fn)
            if val_f1 > best_val_f1:
                print(f"\nValidation F1 improved ({best_val_f1:.4f} --> {val_f1:.4f}). Saving model...")
                best_val_f1 = val_f1; os.makedirs(Config.output_dir, exist_ok=True)
                classifier_model.save_pretrained(Config.output_dir)
                if classifier_tokenizer: classifier_tokenizer.save_pretrained(Config.output_dir)
                print(f"Model saved to {Config.output_dir}")
            else:
                print(f"\nValidation F1 did not improve ({val_f1:.4f}). Current best: {best_val_f1:.4f}")
        else:
            print("\nSkipping validation: Validation dataloader is empty.")
        gc.collect()
        if Config.device == torch.device("cuda"): torch.cuda.empty_cache()
    print("\n--- Training Finished ---")


    # --- Final Evaluation on Test Set ---
    # (Keep final evaluation as is)
    if len(test_dataloader) > 0:
        print("\n--- Evaluating on Test Set using the Best Model ---")
        try:
            best_model_path = Config.output_dir
            if os.path.exists(best_model_path) and os.path.exists(os.path.join(best_model_path, "pytorch_model.bin")):
                print(f"Loading best model from {best_model_path}...")
                best_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
                best_tokenizer = AutoTokenizer.from_pretrained(best_model_path)
                best_model.to(Config.device)
                print("Re-creating test dataset with loaded tokenizer...")
                test_dataset_final = RagNewsDataset(texts=test_df['tweet'].tolist(), labels=test_df['label'].tolist(), tokenizer=best_tokenizer, max_len=Config.max_seq_length, retriever_func=retrieve_documents, num_retrieved=Config.num_retrieved_docs)
                # Determine num_workers/pin_memory for final test loader
                num_workers_test = 0
                if Config.DEBUG_DATALOADER: num_workers_test = 0 # Keep 0 if debugging
                elif Config.device == torch.device("cuda") and mp.get_start_method(allow_none=True) == 'spawn': num_workers_test = 2
                pin_memory_test = (num_workers_test > 0 and Config.device == torch.device("cuda"))
                test_dataloader_final = DataLoader(test_dataset_final, batch_size=Config.batch_size, shuffle=False, num_workers=num_workers_test, pin_memory=pin_memory_test)
                print("Evaluating best model on the test set...")
                if loss_fn: evaluate_model(best_model, test_dataloader_final, Config.device, loss_fn)
                else: print("Error: Loss function not defined for final evaluation.")
            elif classifier_model:
                 print("No saved best model found. Evaluating using the model's final state...")
                 if loss_fn: evaluate_model(classifier_model, test_dataloader, Config.device, loss_fn)
                 else: print("Error: Loss function not defined for final evaluation.")
            else: print("Skipping final evaluation: No model available.")
        except Exception as e:
            print(f"Error during final test evaluation: {e}"); traceback.print_exc()
    else:
        print("\nSkipping final evaluation: Test dataloader is empty.")

    print("\n--- Script Finished ---")


Multiprocessing start method set to 'spawn' (was None).
CUDA available. Using device: cuda

--- Setting up Retriever ---
Loaded 6420 samples from ../Constraint_English_Train.xlsx
Loading retriever model: all-MiniLM-L6-v2...
Retriever model loaded.
Embedding 6420 training documents...


Batches:   0%|          | 0/101 [00:00<?, ?it/s]

Embeddings shape: torch.Size([6420, 384])
Building FAISS index (IndexFlatL2) with dimension 384...
FAISS index built with 6420 vectors.

--- Loading Data and Creating DataLoaders ---
Loaded 2140 samples from ../Constraint_English_Val.xlsx
Loaded 2140 samples from ../english_test_with_labels.xlsx
Loading classifier tokenizer: bert-base-uncased...
Classifier tokenizer loaded.
Creating datasets...
Datasets created.
DEBUG MODE: Setting num_workers = 0 to get detailed error tracebacks.
pin_memory set to: False
DataLoaders created.
  Train batches: 803
  Validation batches: 268
  Test batches: 268

--- Initializing Classifier Model ---
Loading classifier model: bert-base-uncased...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Classifier model loaded and moved to device.

--- Setting up Training Components ---
Optimizer, Scheduler, and Loss Function initialized.
Total training steps (considering grad accum): 2409

--- Starting Training ---

===== Epoch 1/3 =====


...


Epoch 3 completed in 227.76 seconds.
  Average Training Loss: 0.0092

--- Evaluating on Validation Set ---


Evaluating:   0%|          | 0/268 [00:00<?, ?it/s]


Evaluation Results:
  Loss: 0.1993
  Accuracy: 0.9584
  Precision: 0.9640
  Recall: 0.9563
  F1-Score: 0.9601

Validation F1 improved (0.9566 --> 0.9601). Saving model...
Model saved to ./rag_classifier_model

--- Training Finished ---

--- Evaluating on Test Set using the Best Model ---
No saved best model found. Evaluating using the model's final state...


Evaluating:   0%|          | 0/268 [00:00<?, ?it/s]


Evaluation Results:
  Loss: 0.2083
  Accuracy: 0.9617
  Precision: 0.9634
  Recall: 0.9634
  F1-Score: 0.9634

--- Script Finished ---
